# Zero to Hero

## 1.04 Transformado (innecesariamente) las variables

El objetivo de esta sección es analizar el efecto que tiene sobre el arbol de decision
* Variables Colineales
* Normalizacion de Variables
* Transformada logarítmica
* Outliers

In [ ]:
rm( list=ls())
gc()

library( "data.table")   #cargo la libreria  data.table
library( "rpart")  #cargo la libreria  rpart
library( "rpart.plot")

options(repr.plot.width=20, repr.plot.height=10) 
setwd("d:\\gdrive\\Austral2022r\\")  #Aqui se debe poner la ruta de la PC local

In [ ]:
dataset <- fread("./datasets/paquete_premium_202011.csv")   #cargo el dataset

In [ ]:
#genero el modelo
modelo <-  rpart(  formula= "clase_ternaria ~ ." ,
                   data= dataset,
                   xval= 0,
                   cp= -1,
                   maxdepth= 2 )

#imprimo el modelo graficamente
prp(modelo, extra=101, digits=5, branch=1, type=4, varlen=0, faclen=0, tweak=1.1, cex=1.2)

La variable mas importante que aparece es **ctrx_quarter**

### variables colineales

agrego al dataset tres variables colineales  con ctrx_quarter

In [ ]:
dataset[  , ctrx_quarter_dos    :=  2*ctrx_quarter ]
dataset[  , ctrx_quarter_tres   :=  3*ctrx_quarter ]
dataset[  , ctrx_quarter_cuatro :=  4*ctrx_quarter ]

y vuelvo a correr el arbol de decision

In [ ]:
#genero el modelo
modelo <-  rpart(  formula= "clase_ternaria ~ ." ,
                   data= dataset,
                   xval= 0,
                   cp= -1,
                   maxdepth= 2 )

#imprimo el modelo graficamente
prp(modelo, extra=101, digits=5, branch=1, type=4, varlen=0, faclen=0, tweak=1.1, cex=1.2)

**SORPRENDENTE , el arbol de decision es inmune a las colinearidad de variables**
<br>
El arbol de decisión no ha cambiado. Su forma es exactamente igual, las cantidades en los nodos idéntica al arbol original.

###  Normalizacion de variables

Analizo la variable  ctrx_quarter

In [ ]:
min( dataset[ , ctrx_quarter] )

In [ ]:
max( dataset[ , ctrx_quarter] )

In [ ]:
boxplot(  dataset[ , ctrx_quarter])

In [ ]:
hist( dataset[, ctrx_quarter] )

In [ ]:
plot( density( dataset[, ctrx_quarter] ) )

Normailzo  ctrx_quarter

In [ ]:
dataset[ , ctrx_quarter := scale(ctrx_quarter)]

confirmo que me quedó normalizada

In [ ]:
plot( density( dataset[, ctrx_quarter] ) )

Confirmado, la variable está normalizada, ahora corremos nuevamente el arbol de decision

In [ ]:
#genero el modelo
modelo <-  rpart(  formula= "clase_ternaria ~ ." ,
                   data= dataset,
                   xval= 0,
                   cp= -1,
                   maxdepth= 2 )

#imprimo el modelo graficamente
prp(modelo, extra=101, digits=5, branch=1, type=4, varlen=0, faclen=0, tweak=1.1, cex=1.2)

**SORPRENDENTE , el arbol de decision es inmune a las normalizacion de variables**
<br>
El arbol de decisión no ha cambiado. Su forma es exactamente igual, las cantidades en los nodos idéntica al arbol original.
<br>
Por supuesto, el arbol original cortaba en  ctrx_quarter < 27 y ahora corta en ctrx_quarter < -0.98119 porque obviamente alteré esa variable, pero en realidad está cortando en el mismo punto.

### Transformación  logaritmica

Primero vuelvo a cargar el dataset

In [ ]:
dataset <- fread("./datasets/paquete_premium_202011.csv")   #cargo el dataset

hago la transformación logarítmica

In [ ]:
dataset[ , ctrx_quarter :=log(ctrx_quarter+1)]  #sumo el uno porque no quiero infinitos

Ahora veo el boxplot

In [ ]:
boxplot(  dataset[ , ctrx_quarter])

In [ ]:
plot( density( dataset[, ctrx_quarter] ) )

Finalmente, el arbol de decision

In [ ]:
#genero el modelo
modelo <-  rpart(  formula= "clase_ternaria ~ ." ,
                   data= dataset,
                   xval= 0,
                   cp= -1,
                   maxdepth= 2 )

#imprimo el modelo graficamente
prp(modelo, extra=101, digits=5, branch=1, type=4, varlen=0, faclen=0, tweak=1.1, cex=1.2)

**SORPRENDENTE , el arbol de decision es inmune a la transformada logaritmica**
<br>
El arbol de decisión no ha cambiado. Su forma es exactamente igual, las cantidades en los nodos idéntica al arbol original.
<br>
Por supuesto, el arbol original cortaba en  ctrx_quarter < 27 y ahora corta en ctrx_quarter < 3.314 porque obviamente alteré esa variable, pero en realidad está cortando en el mismo punto.

### Outliers

Ahora fabrico outliers y veo como se comporta el arbol

Primero vuelvo a cargar el dataset

In [ ]:
dataset <- fread("./datasets/paquete_premium_202011.csv")   #cargo el dataset

el boxplot original

In [ ]:
boxplot(  dataset[ , ctrx_quarter])

cuento cuantos registros hay con ctrx_quarter > 1500

In [ ]:
dataset[ ctrx_quarter > 1500, .N]

Ahora, a esos 16 valores los transformo en outliers extremos

In [ ]:
dataset[ ctrx_quarter > 1500,  ctrx_quarter := ctrx_quarter * 1000]

compruebo que sean outliers extremos

In [ ]:
boxplot(  dataset[ , ctrx_quarter])

Finalmente, nuevamente arbol de decision

In [ ]:
#genero el modelo
modelo <-  rpart(  formula= "clase_ternaria ~ ." ,
                   data= dataset,
                   xval= 0,
                   cp= -1,
                   maxdepth= 2 )

#imprimo el modelo graficamente
prp(modelo, extra=101, digits=5, branch=1, type=4, varlen=0, faclen=0, tweak=1.1, cex=1.2)

**SORPRENDENTE, el arbol de decision es inmune a los outliers**
<br>
El arbol de decisión no ha cambiado. Su forma es exactamente igual, las cantidades en los nodos idéntica al arbol original.
<br>

Si usted que fueron muy conservador pasar tan solo 16 valor a que sean outliers, pruebe con  ctrx_quarter > 1000